# Combine sources

## Setup

In [1]:
import pathlib 

import pandas as pd
import numpy as np

## Reading raw data

### 1st source (https://www.kaggle.com/datasets/robikscube/flight-delay-dataset-20182022)

In [2]:
source_1_dir: pathlib.Path = pathlib.Path('../datasets/raw/source_1/')

parquet_files: list[pathlib.Path] = list(source_1_dir.glob("Combined_Flights_[0-9]*.parquet"))

source_1_df: pd.DataFrame = pd.concat((pd.read_parquet(parquet_file) for parquet_file in parquet_files), ignore_index=True)

#### Cleaning

City name is in the format "city,state", we remove the state part because it is given by another column.

In [3]:
source_1_df['DestCityName'] = source_1_df['DestCityName'].apply(lambda x: x.split(',')[0])
source_1_df['OriginCityName'] = source_1_df['OriginCityName'].apply(lambda x: x.split(',')[0])

### Join with additional airport information
Since the airport code assigned by IATA can change over time and can be reused, we use the AirportID codes assigned by US DOT to aquire airport names.

Source: https://www.transtats.bts.gov/Fields.asp?gnoyr_VQ=GED

In [4]:
source_1_airports: pd.DataFrame = pd.read_csv(source_1_dir / 'L_AIRPORT_ID.csv').set_index('Code')

In [5]:
source_1_df['OriginAirport'] = source_1_df['OriginAirportID'].map(source_1_airports['Description'].apply(lambda x: x.split(':')[-1]))
source_1_df['DestAirport'] = source_1_df['DestAirportID'].map(source_1_airports['Description'].apply(lambda x: x.split(':')[-1]))

### 2nd source (https://community.amstat.org/jointscsg-section/dataexpo/dataexpo2009)


In [6]:
source_2_dir: pathlib.Path = pathlib.Path('../datasets/raw/source_2/')

csv_files: list[pathlib.Path] = list(source_2_dir.glob("[0-9]*.csv"))

source_2_df: pd.DataFrame = pd.concat((pd.read_csv(csv_file, encoding='latin-1') for csv_file in csv_files), ignore_index=True)

/var/folders/c9/24l9wbvn7_l_zt25742x5qf40000gn/T/ipykernel_15366/3230565564.py:5: DtypeWarning: Columns (22) have mixed types. Specify dtype option on import or set low_memory=False.
  source_2_df: pd.DataFrame = pd.concat((pd.read_csv(csv_file, encoding='latin-1') for csv_file in csv_files), ignore_index=True)


#### Joining with airport information

In [7]:
source_2_airports_df: pd.DataFrame = pd.read_csv(source_2_dir / 'airports.csv', encoding='latin-1')
source_2_df = source_2_df.merge(source_2_airports_df[['iata', 'airport', 'city', 'state']], left_on='Origin', right_on='iata', how='left', suffixes=('', '_origin'))
source_2_df = source_2_df.rename({
    'iata': 'OriginIata', 
    'airport': 'OriginAirport', 
    'city': 'OriginCityName', 
    'state': 'OriginState'}, axis=1)
source_2_df = source_2_df.merge(source_2_airports_df[['iata', 'airport', 'city', 'state']], left_on='Dest', right_on='iata', how='left', suffixes=('', '_dest'))
source_2_df = source_2_df.rename({
    'iata': 'DestIata',
    'airport': 'DestAirport',
    'city': 'DestCityName',
    'state': 'DestState'}, axis=1)

### Joining with airline information

In [8]:
source_2_carriers_df: pd.DataFrame = pd.read_csv(source_2_dir / 'carriers.csv', encoding='latin-1')
source_2_carriers_df.set_index('Code', inplace=True)

source_2_df['Airline'] = source_2_df['UniqueCarrier'].apply(lambda x: source_2_carriers_df.loc[x, 'Description'])

#### Renaming features to match the first source

In [9]:
source_2_df.rename({'TailNum': 'Tail_Number'}, axis=1, inplace=True)

## Features unique to sources
The following features are unique to the sources, and therefore will not be in the final dataset.

In [10]:
non_overlapping_features: set[str] = set(source_1_df.columns) ^ set(source_2_df.columns)
print("\n".join(non_overlapping_features))

Flight_Number_Operating_Airline
IATA_Code_Marketing_Airline
WheelsOff
FlightNum
DestAirportID
OriginWac
DestAirportSeqID
DOT_ID_Operating_Airline
DestStateFips
OriginStateFips
WheelsOn
ArrivalDelayGroups
UniqueCarrier
ArrTimeBlk
DestWac
OriginStateName
DestIata
CancellationCode
OriginAirportID
CarrierDelay
Marketing_Airline_Network
DivAirportLandings
FlightDate
SecurityDelay
OriginIata
DepTimeBlk
DOT_ID_Marketing_Airline
Flight_Number_Marketing_Airline
DepDelayMinutes
LateAircraftDelay
DepartureDelayGroups
OriginAirportSeqID
DistanceGroup
DestCityMarketID
Operated_or_Branded_Code_Share_Partners
DepDel15
ArrDelayMinutes
NASDelay
Operating_Airline
WeatherDelay
Quarter
OriginCityMarketID
IATA_Code_Operating_Airline
ArrDel15
DestStateName


## Features present in both sources

In [11]:
overlapping_features: set[str] = set(source_1_df.columns) & set(source_2_df.columns)
print("\n".join(overlapping_features))

DayOfWeek
DepDelay
DepTime
TaxiIn
Origin
AirTime
DayofMonth
CRSDepTime
ArrTime
Diverted
Airline
Distance
Tail_Number
Cancelled
ActualElapsedTime
OriginCityName
OriginState
CRSElapsedTime
DestCityName
Month
ArrDelay
DestAirport
DestState
TaxiOut
Dest
OriginAirport
Year
CRSArrTime


## Combining sources

In [12]:
data_df: pd.DataFrame = pd.concat([source_1_df[list(overlapping_features)], source_2_df[list(overlapping_features)]], ignore_index=True)

data_df

,DayOfWeek,DepDelay,DepTime,TaxiIn,Origin,AirTime,DayofMonth,CRSDepTime,ArrTime,Diverted,...,DestCityName,Month,ArrDelay,DestAirport,DestState,TaxiOut,Dest,OriginAirport,Year,CRSArrTime
0,2,-5.0,1157.0,7.0,ABY,38.0,23,1202,1256.0,False,...,Atlanta,1,-8.0,Hartsfield-Jackson Atlanta International,GA,14.0,ATL,Southwest Georgia Regional,2018,1304
1,3,-5.0,1157.0,12.0,ABY,36.0,24,1202,1258.0,False,...,Atlanta,1,-6.0,Hartsfield-Jackson Atlanta International,GA,13.0,ATL,Southwest Georgia Regional,2018,1304
2,4,-9.0,1153.0,11.0,ABY,40.0,25,1202,1302.0,False,...,Atlanta,1,-2.0,Hartsfield-Jackson Atlanta International,GA,18.0,ATL,Southwest Georgia Regional,2018,1304
3,5,-12.0,1150.0,11.0,ABY,35.0,26,1202,1253.0,False,...,Atlanta,1,-11.0,Hartsfield-Jackson Atlanta International,GA,17.0,ATL,Southwest Georgia Regional,2018,1304
4,6,-5.0,1355.0,11.0,ABY,36.0,27,1400,1459.0,False,...,Atlanta,1,-1.0,Hartsfield-Jackson Atlanta International,GA,17.0,ATL,Southwest Georgia Regional,2018,1500
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
148108235,2,60.0,1858.0,NaN,CVG,NaN,12,1758,2035.0,0,...,Philadelphia,12,63.0,Philadelphia Intl,PA,NaN,PHL,Cincinnati Northern Kentucky Intl,1989,1932
148108236,3,9.0,1807.0,NaN,CVG,NaN,13,1758,1945.0,0,...,Philadelphia,12,13.0,Philadelphia Intl,PA,NaN,PHL,Cincinnati Northern Kentucky Intl,1989,1932
148108237,4,0.0,1758.0,NaN,CVG,NaN,14,1758,1923.0,0,...,Philadelphia,12,-9.0,Philadelphia Intl,PA,NaN,PHL,Cincinnati Northern Kentucky Intl,1989,1932
148108238,5,10.0,1810.0,NaN,CVG,NaN,15,1800,1937.0,0,...,Philadelphia,12,7.0,Philadelphia Intl,PA,NaN,PHL,Cincinnati Northern Kentucky Intl,1989,1930


## Convert column types

In [13]:
data_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 148108240 entries, 0 to 148108239
Data columns (total 28 columns):
 #   Column             Dtype  
---  ------             -----  
 0   DayOfWeek          int64  
 1   DepDelay           float64
 2   DepTime            float64
 3   TaxiIn             float64
 4   Origin             object 
 5   AirTime            float64
 6   DayofMonth         int64  
 7   CRSDepTime         int64  
 8   ArrTime            float64
 9   Diverted           object 
 10  Airline            object 
 11  Distance           float64
 12  Tail_Number        object 
 13  Cancelled          object 
 14  ActualElapsedTime  float64
 15  OriginCityName     object 
 16  OriginState        object 
 17  CRSElapsedTime     float64
 18  DestCityName       object 
 19  Month              int64  
 20  ArrDelay           float64
 21  DestAirport        object 
 22  DestState          object 
 23  TaxiOut            float64
 24  Dest               object 
 25  OriginAirport 

### Boolean columns

We suspect that the columns Diverted and Cancelled only contain boolean values.
We confirm it by using `value_counts()` and convert them to boolean type.

In [14]:
data_df['Diverted'].value_counts()

False    147767298
True        340942
Name: Diverted, dtype: int64

In [15]:
data_df['Cancelled'].value_counts()

False    145100641
True       3007599
Name: Cancelled, dtype: int64

In [16]:
boolean_columns: list[str] = ['Diverted', 'Cancelled']

for column in boolean_columns:
    data_df[column] = data_df[column].astype("bool")

In [17]:
data_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 148108240 entries, 0 to 148108239
Data columns (total 28 columns):
 #   Column             Dtype  
---  ------             -----  
 0   DayOfWeek          int64  
 1   DepDelay           float64
 2   DepTime            float64
 3   TaxiIn             float64
 4   Origin             object 
 5   AirTime            float64
 6   DayofMonth         int64  
 7   CRSDepTime         int64  
 8   ArrTime            float64
 9   Diverted           bool   
 10  Airline            object 
 11  Distance           float64
 12  Tail_Number        object 
 13  Cancelled          bool   
 14  ActualElapsedTime  float64
 15  OriginCityName     object 
 16  OriginState        object 
 17  CRSElapsedTime     float64
 18  DestCityName       object 
 19  Month              int64  
 20  ArrDelay           float64
 21  DestAirport        object 
 22  DestState          object 
 23  TaxiOut            float64
 24  Dest               object 
 25  OriginAirport 

### Categorical columns

In [18]:
categorical_columns: list[str] = ['DayOfWeek', 'OriginCityName', 'OriginState', 'OriginAirport',
                                  'DestState', 'Tail_Number', 'Dest', 'DayofMonth', 'DestAirport',
                                  'Airline', 'DestCityName', 'Origin', 'Year', 'Month']
                                
for column in categorical_columns:
    data_df[column] = data_df[column].astype("category")

In [19]:
data_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 148108240 entries, 0 to 148108239
Data columns (total 28 columns):
 #   Column             Dtype   
---  ------             -----   
 0   DayOfWeek          category
 1   DepDelay           float64 
 2   DepTime            float64 
 3   TaxiIn             float64 
 4   Origin             category
 5   AirTime            float64 
 6   DayofMonth         category
 7   CRSDepTime         int64   
 8   ArrTime            float64 
 9   Diverted           bool    
 10  Airline            category
 11  Distance           float64 
 12  Tail_Number        category
 13  Cancelled          bool    
 14  ActualElapsedTime  float64 
 15  OriginCityName     category
 16  OriginState        category
 17  CRSElapsedTime     float64 
 18  DestCityName       category
 19  Month              category
 20  ArrDelay           float64 
 21  DestAirport        category
 22  DestState          category
 23  TaxiOut            float64 
 24  Dest               c

## Save the combined dataset

In [20]:
data_df.to_parquet('../datasets/raw/combined.parquet')

: 